<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

<a id="section-one"></a>
# Задача

Прогнозирование стоимости автомобиля по характеристикам

Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE

В данном ноутбуке мы загрузили датасет, который был подготовлен на предыдущих этапах. 
Также в этом проекте мы использовали:
- 1этап:
    Ноутбук, через который парсили https://www.kaggle.com/chvassilissa/vassilissa-car-price-prediction-part1
    
    Спарсенный датасет 
- 2 этап:
    Ноутбук, в котором провели EDA https://www.kaggle.com/chvassilissa/vassilissa-car-price-prediction-part2
    
    Датасет, после EDA  https://www.kaggle.com/chvassilissa/parser-auto-012022
- 3 этап:
    Ноутбук, в котором провели обучение https://www.kaggle.com/chvassilissa/vassilissa-car-price-prediction-part3

In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import ipywidgets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from catboost import CatBoostRegressor
from catboost import cv, Pool

#from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
#from hyperopt.pyll.base import scop

from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor

import xgboost as xgb

from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

In [48]:
#!jupyter nbextension enable --py widgetsnbextension

In [49]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [50]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [51]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [52]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [53]:
VERSION    = 16
DIR_TRAIN  = '../input/parser-auto-012022/'# подключила к ноутбуку сет с пред шага
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [54]:
!ls '../input'

In [55]:
db_source = pd.read_csv(DIR_TRAIN+'cdf_auto.csv') # датасет db_source содержит 2 базы train (train = 1) и test  (train = 0)
# test = pd.read_csv(DIR_TEST+'test.csv') #
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

#Выбранные признаки в EDA
data_columns = pd.read_csv(DIR_TRAIN+'cdf_auto_columns.csv') 

cat_cols = data_columns[data_columns.column_type == 'cat'].column_name.values.tolist()
bin_cols = data_columns[data_columns.column_type == 'bin'].column_name.values.tolist()
num_cols = data_columns[data_columns.column_type == 'num'].column_name.values.tolist()
target_cols = data_columns[data_columns.column_type == 'target'].column_name.values.tolist()
add_cols = data_columns[data_columns.column_type == 'add'].column_name.values.tolist()


In [56]:
db_source

In [57]:
print("Трейн-данных: ", len(db_source[db_source.train == 1]))
print("Тест-данных: ", len(db_source[db_source.train == 0]))
print("Кол-во столбцов: ", len(db_source.columns))
#Трейн-данных:  133055
#Тест-данных:  34686
#Кол-во столбцов:  32

In [58]:
# датафрейм для сбора результатов по каждому экспериметну
#df_Result = pd.DataFrame(columns=['model', 'MAPE'])

## Label Encoding

In [59]:
db_le = db_source.copy()
for colum in cat_cols + bin_cols:
    db_le[colum] = db_source[colum].astype('category').cat.codes
# db_le = database_Label Encoding

In [60]:
db_le

In [61]:
X = db_le.query('train == 1').drop(add_cols + target_cols, axis=1)#train
X_sub = db_le.query('train == 0').drop(add_cols + target_cols, axis=1) #test
y = db_source[db_source.train == 1]['price'] # train_target

In [62]:
print("Трейн-данных: ", len(X))
print("Тест-данных: ", len(X_sub))
print("Трейн-целевая: ", len(y))
# Трейн-данных:  133055
#Тест-данных:  34686
#Трейн-целевая:  133055

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
# X_train, X_test, y_train, y_test - только Label Encoding
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## + MinMaxScaler

In [64]:
db_sc = db_le.copy()
db_sc[num_cols] = pd.DataFrame(MinMaxScaler().fit_transform(db_sc[num_cols].values), \
                                     columns = db_sc[num_cols].columns)

In [65]:
X_sc = db_sc.query('train == 1').drop(add_cols + target_cols, axis=1)#train
X_sc_sub = db_sc.query('train == 0').drop(add_cols + target_cols, axis=1) #test

In [66]:
# X_train_sc, X_test_sc, y_train_sc, y_test_sc - Label Encoding + MinMaxScaler
X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(X_sc, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
print(X_train_sc.shape, X_test_sc.shape, y_train_sc.shape, y_test_sc.shape)

## + Dummies

In [67]:
db_sc_dumm = pd.get_dummies(db_sc, columns = cat_cols, dummy_na=False)

In [68]:
X_sc_dumm = db_sc_dumm.query('train == 1').drop(add_cols + target_cols, axis=1)#train
X_sc_dumm_sub = db_sc_dumm.query('train == 0').drop(add_cols + target_cols, axis=1) #test

In [69]:
# X_train_sc_dumm, X_test_sc_dumm, y_train_sc_dumm, y_test_sc_dumm - Label Encoding + MinMaxScaler+dummies
X_train_sc_dumm, X_test_sc_dumm, y_train_sc_dumm, y_test_sc_dumm = train_test_split(X_sc_dumm, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)
print(X_train_sc_dumm.shape, X_test_sc_dumm.shape, y_train_sc_dumm.shape, y_test_sc_dumm.shape)

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (enginePower). 
C ней будем сравнивать другие модели.




In [70]:
"""tmp_train = X_train.copy()
tmp_train['price'] = y_train"""

In [71]:
"""# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['enginePower'].map(tmp_train.groupby('enginePower')['price'].median())

#оцениваем точность
result = round((mape(y_test, predict.values))*100,2)
df_Result.loc[len(df_Result)] = ['naiv', result]
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")
# Точность наивной модели по метрике MAPE: 60.66%"""

# Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

**Полезные видео о CatBoost (на русском):**
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Default - 15.06%

In [72]:
"""catb_def = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
catb_def.fit(X_train, y_train,
         #cat_features= cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

# оцениваем точность
predict_test = catb_def.predict(X_test)

result = round((mape(y_test, predict_test))*100,2)
df_Result.loc[len(df_Result)] = ['catb_def', result]

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
# Точность модели по метрике MAPE: 15.06%"""

## Log Target - 12.70%
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    

In [73]:
"""catb_log = CatBoostRegressor(iterations = 5000,
                                   random_seed = RANDOM_SEED,
                                   eval_metric='MAPE', # улучшаемая метрика
                                   custom_metric=['R2', 'MAE'], # метрика для отображения на графике
                                   silent=True,
                                  )
catb_log.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

predict_test = np.exp(catb_log.predict(X_test))
result = round((mape(y_test, predict_test))*100,2)
print(f"Точность модели по метрике MAPE: {result}%")

#Точность модели по метрике MAPE: 12.48%
#predict_submission = np.exp(catb_log.predict(X_sub))
#Точность модели по метрике MAPE: 12.7% на лидерборд 13.56549"""

## Log_AdvPar - 12.64%

In [74]:
"""catb_log_adv = CatBoostRegressor(iterations = 6000,
                                   random_seed = RANDOM_SEED,
                                   eval_metric='MAPE', # улучшаемая метрика
                                   custom_metric=['R2', 'MAE'],
                                   silent=True,
                                   learning_rate = 0.05,
                                   l2_leaf_reg = 0, # Coefficient at the L2 
                                   od_type = 'Iter', # детектор переобучения
                                   od_wait = 20 # детектор переобучения
                                  )
catb_log_adv.fit(X_train, np.log(y_train),
         # cat_features=cat_cols + bin_cols, #12.78%
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True         \
        )
                 
print(catb_log_adv.tree_count_)

predict_test = np.exp(catb_log_adv.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")
predict_submission = np.exp(catb_log_adv.predict(X_sub))
# Точность модели по метрике MAPE: 12.64%  на лидерборд 13.48127"""

In [75]:
# learning_rate = 0.05, l2_leaf_reg = 0, 12.64%
# learning_rate = 0.05,  l2_leaf_reg = 5, 12.64%  4431 Точность модели по метрике MAPE: 12.68%
# learning_rate = 0.05, l2_leaf_reg = 7 12.84%
# learning_rate = 0.03, l2_leaf_reg = 0 - 6000 Точность модели по метрике MAPE: 12.71%


In [76]:
"""cat_cur_imp = pd.Series(catb_log_adv.get_feature_importance(),
                         X_test.columns)
fig, ax = plt.subplots(figsize=(16,14))
cat_cur_imp.plot.bar(ax=ax)
ax.set_title("Важность признаков")
fig.tight_layout()"""

## Log_AdvPar_KFold - 12.55%

In [77]:
"""catb_log_adv_kf = CatBoostRegressor(
                                  iterations = 5000,
                                   random_seed = RANDOM_SEED,
                                   eval_metric='MAPE', # улучшаемая метрика
                                   custom_metric=['R2', 'MAE'],
                                   silent=True,
                                   learning_rate = 0.1,
                                   l2_leaf_reg = 7, # Coefficient at the L2 
                                   od_type = 'Iter', # детектор переобучения
                                   od_wait = 20 # детектор переобучения
                          )

skf = KFold(n_splits=4, random_state=RANDOM_SEED, shuffle=True)
cat_boost_log_mape_values = []

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    
    catb_log_adv_kf.fit(X_train, np.log(y_train),
          #cat_features=cat_features_ids,
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=0,
          use_best_model=True,
          #plot=True
          )

    y_pred = np.exp(catb_log_adv_kf.predict(X_test))

    cat_boost_log_mape_value = mape(y_test, y_pred)
    cat_boost_log_mape_values.append(cat_boost_log_mape_value)
    print(cat_boost_log_mape_value)

print(f"Точность модели по метрике MAPE: {(np.mean(cat_boost_log_mape_values) * 100):0.2f}%.")
result = round((np.mean(cat_boost_log_mape_values) * 100),2)
df_Result.loc[len(df_Result)] = ['catb_log_adv_kf', result]
predict_submission = np.exp(catb_log_adv_kf.predict(X_sub))

#Точность модели по метрике MAPE: 12.55%. на лидерборд  13.62093 % 0.7"""

In [78]:
#Эксперименты с коэф инфляции по скаченной базе (в январе 2022). Лучший результат 30%. 
# Как видно коэф инфляции сильно влияет на точность предсказания
#Точность модели по метрике MAPE: 12.64%. на лидерборд 14.87308 % 0.65
# Точность модели по метрике MAPE: 12.54%. на лидерборд  13.79496 # 0.68
# Точность модели по метрике MAPE: 12.58%. на лидерборд  13.8187 # 0.69
#Точность модели по метрике MAPE: 12.55%. на лидерборд  13.62093 % 0.7
# Точность модели по метрике MAPE: 12.61%. на лидерборд 14.08865 % 0.71
#Точность модели по метрике MAPE: 12.61%. на лидерборд 15.60842 %0.75

Промежуточные выводы по модели CatBoostRegressor:
1. С дефолтными параметрами на базе после LabelEncoding дает результат 15.06%
2. Логорифмирование целевой базы улучшило обучение +2.36% и дошло до 12.70%. В дальнейшем буду использовать обучение с логорифмированной целевой переменной;
3. Подбор параметров немного улучшил точность +0.16% и дошло до 12.64. Подкручены были параметры:  learning_rate = 0.05, l2_leaf_reg = 0, # Coefficient at the L2, od_type = 'Iter', # детектор переобучения
   od_wait = 20 # детектор переобучения
4. Самое интересное что для CatBoostRegressor такие штуки как LabelEncoding, MinMaxScaler, getDummies, KFold не дали никакого улучшения. 
5. Таким образом если датасет содержит большое количество категориальных данных можно после EDA+FE сразу подложить в обучение, подкрутив параметры качества (скорости) в зависимости от цели.
6. Лучший резльтат на каггле дате вариант с подкрученными параметрами. Где точность модели по метрике MAPE: 12.64%  на лидерборд 13.48127

# Model 3 : RandomForestRegressor
Целевая переменная в дальнейшем всегда логорифмирована

## DefPar -13.41%.

In [79]:
"""rf_def = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_def.fit(X_train, np.log(y_train))
predict = np.exp(rf_def.predict(X_test))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict) * 100):0.2f}%.")
result = round((mape(y_test, predict_rf_def) * 100),2)

# Точность модели по метрике MAPE: 13.41%."""

## AdvPar - 13.74%

In [80]:
"""# Подбор параметров для Random forest
# Долго работает и не дает улучшений
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2001, num = 50)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 111, num = 20)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4]
              }
              
rf_adv = RandomForestRegressor(random_state=RANDOM_SEED)
rf_random = RandomizedSearchCV(estimator=rf_adv, param_distributions=random_grid, n_iter=100, 
                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
rf_random.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf_random.predict(X_test))))*100:0.2f}%")

#-------------------------------
model = rf_random.best_estimator_
best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(model.predict(X_test))))*100:0.2f}%")"""


In [81]:
# Долго работает и не дает улучшений
"""print(rf_random.best_params_)
print(rf_random.best_score_)
print(rf_random.best_estimator_)
#{'n_estimators': 1927, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 25}
#0.9111820501392165
#RandomForestRegressor(max_depth=25, n_estimators=1927, random_state=42)"""
#----------------

In [82]:
"""# данные после LabelEncoding + AdvPar
rf_adv = RandomForestRegressor(max_depth=25, n_estimators=1927, random_state=RANDOM_SEED)
rf_adv.fit(X_train, np.log(y_train))
predict = np.exp(rf_adv.predict(X_test))

result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")

predict_submission = np.exp(rf_adv.predict(X_sub))

# Точность модели по метрике MAPE: 13.31%."""

In [83]:
"""# данные после LabelEncoding,  MinMaxScaler, getDummies + AdvPar
rf_adv = RandomForestRegressor(max_depth=25, n_estimators=1927, random_state=42)
rf_adv.fit(X_train_sc_dumm, np.log(y_train_sc_dumm))
predict = np.exp(rf_adv.predict(X_test_sc_dumm))

result = round((mape(y_test_sc_dumm, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")"""

## DefPar_Stan_Dumm - 13.29 %

In [84]:
"""# данные после LabelEncoding, MinMaxScaler и подобранные параметры
rf_adv_stan = RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      min_samples_split=5, n_estimators=1800, random_state=RANDOM_SEED)
rf_adv_stan.fit(X_train_sc, np.log(y_train_sc))
predict = np.exp(rf_adv_stan.predict(X_test_sc))
result = round((mape(y_test_sc, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
# Точность модели по метрике MAPE: 13.74%."""

In [85]:
"""# данные после LabelEncoding, MinMaxScaler и дефолтные параметры
rf_def_stan = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_def_stan.fit(X_train_sc, np.log(y_train_sc))
predict = np.exp(rf_def_stan.predict(X_test_sc))
result = round((mape(y_test_sc, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
# Точность модели по метрике MAPE: 13.41%."""

In [86]:
"""# данные после LabelEncoding,  MinMaxScaler, getDummies и дефолтные параметры
rf_def_stan_dumm = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_def_stan_dumm.fit(X_train_sc_dumm, np.log(y_train_sc_dumm))
predict = np.exp(rf_def_stan_dumm.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")

predict_submission = np.exp(rf_def_stan_dumm.predict(X_sc_dumm_sub))

# Точность модели по метрике MAPE: 13.29%. на лидерборд 12.56475 """

Промежуточные выводы по модели RandomForestRegressor:
1. Модель показала себя хуже чем CatBoost, лучший результат 13.29%.
2. Хорошо при обучении себя провила даммикодирование (+0.12)
3. При сабмите показал хороший результат RandomForestRegressor на данных после LabelEncoding,  MinMaxScaler, getDummies и дефолтные параметры- 12.56475

# Model 3 - DecisionTreeRegressor

## DefPar - 16.61%

In [87]:
"""dt_def = DecisionTreeRegressor(max_depth=15, random_state = RANDOM_SEED)
dt_def.fit(X_train, np.log(y_train))
predict = np.exp(dt_def.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
predict_submission = np.exp(dt_def.predict(X_sub))
# Точность модели по метрике MAPE: 16.61%. на лидерборде 16.23755"""

## DefPar_Stan -  16.62%

In [88]:
"""# данные стандартизированы, дефолтные параметры
dt_def_sc = DecisionTreeRegressor(max_depth=15, random_state = RANDOM_SEED)
dt_def_sc.fit(X_train_sc, np.log(y_train_sc))
predict = np.exp(dt_def_sc.predict(X_test_sc))
result = round((mape(y_test_sc, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
# Точность модели по метрике MAPE: 16.62%"""

In [89]:
"""# данные стандартизированы + даммизированы, дефолтные параметры
dt_def_sc_dumm = DecisionTreeRegressor(max_depth=15, random_state = RANDOM_SEED)
dt_def_sc_dumm.fit(X_train_sc_dumm, np.log(y_train_sc_dumm))
predict = np.exp(dt_def_sc_dumm.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
# Точность модели по метрике MAPE: 17.01%."""

## AdvPar_Stan

In [90]:
"""parameters = {'splitter':('best','random'), 
              'max_depth':np.arange(10,20), 
              'min_samples_split':np.arange(2,10), 
              'min_samples_leaf':np.arange(2,10)}

regressor = GridSearchCV(DecisionTreeRegressor(), parameters, cv = 3)
regressor.fit(X_train_sc, np.log(y_train_sc+1))

print('Best score: ', regressor.best_score_)
print(regressor.best_params_)
dt_adv_sc = regressor.best_estimator_
predict_dt_adv_sc = np.exp(dt_adv_sc.predict(X_test_sc))
result = round((mape(y_test_sc, predict_dt_adv_sc) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%.")
print(regressor.best_params_)
"""

Промежуточные выводы по DecisionTreeRegressor:
1. Точность предсказания плохая по сравнению с другими моделями.
2. Стандартизирование не улучшило ситуацию, даммизирование даже ухудшило.
3. Подбор лучших параметров работает долго, не буду дальше улучшать эту модель..

# Model 4 - ExtraTreesRegressor

## DefPar - 13.72%

In [91]:
"""etr_def = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_def.fit(X_train, np.log(y_train+1))
predict = np.exp(etr_def.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.72%"""

## DefPar_Stan - 13.5%

In [92]:
"""# данные стандартизированы, дефолтные параметры
etr_def = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_def.fit(X_train_sc, np.log(y_train_sc+1))
predict = np.exp(etr_def.predict(X_test_sc))
result = round((mape(y_test_sc, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.72%"""

In [93]:
"""# данные стандартизированы + даммизированы, дефолтные параметры
etr_def = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_def.fit(X_train_sc_dumm, np.log(y_train_sc_dumm+1))
predict_etr_def = np.exp(etr_def.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict_etr_def) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")

predict_submission = np.exp(etr_def.predict(X_sc_dumm_sub))
# Точность модели по метрике MAPE: 13.5% на лидерборде 12.391"""

## AdvPar

In [ ]:
"""
# 1 вариант подбора параметров
space_etr = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_samples_split': hp.choice('min_samples_split',np.arange(2,10, 1, dtype=int)),
            'min_samples_leaf': hp.choice('min_samples_leaf',np.arange(1,5, 1, dtype=int)),
             }

def hyperopt_etr_score(params):
    clf = ExtraTreesRegressor(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    # для ускорения можно использовать train_test_split один раз
    current_score = cross_val_score(clf, X, y, cv=3).mean()
    print(current_score, params)
    return -current_score
 
 
simple_space_etr = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_samples_split': hp.choice('min_samples_split',np.arange(2,10, 1, dtype=int)),
            'min_samples_leaf': hp.choice('min_samples_leaf',np.arange(1,5, 1, dtype=int)),
               }
 
best = fmin(fn=hyperopt_etr_score, space=simple_space_etr, algo=tpe.suggest, max_evals=2)
best
"""

In [ ]:
"""
# 2 вариант подбора параметров
def objective(params):
     model=ExtraTreesRegressor(
         n_estimators=int(params['n_estimators']),
         max_depth=int(params['max_depth']),
         min_samples_leaf=int(params['min_samples_leaf']),
         min_samples_split=int(params['min_samples_split']),
         bootstrap=params['bootstrap'],
         max_features=params['max_features'],
         random_state=RANDOM_SEED,
         n_jobs=-1
     )
     model.fit(X_train, np.log(y_train))
     pred=model.predict(X_test)
     score=mape(y_test,np.exp(pred))
     return score

def optimize(trial):
    params={
         'n_estimators': hp.choice('n_estimators', range(100, 500)),
         'max_features': hp.choice("max_features", ['auto', 'sqrt']),
         'max_depth': hp.choice('max_depth',np.arange(5,15, dtype=int)),
          'min_samples_split': hp.choice('min_samples_split',np.arange(2,10, 1, dtype=int)),
          'min_samples_leaf': hp.choice('min_samples_leaf',np.arange(1,5, 1, dtype=int)),
         'bootstrap': hp.choice("bootstrap", [True, False])
     }

            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_samples_split': hp.choice('min_samples_split',np.arange(2,10, 1, dtype=int)),
            'min_samples_leaf': hp.choice('min_samples_leaf',np.arange(1,5, 1, dtype=int)),
                
    best=fmin(fn=objective, space=params, algo=tpe.suggest, trials=trial, max_evals=20, rstate=np.random.RandomState(RANDOM_SEED))
    return best

trial=Trials()
best=optimize(trial)
"""

In [113]:
"""etr_adv = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, max_depth = 10, n_estimators= 206)
etr_adv.fit(X_train_sc_dumm, np.log(y_train_sc_dumm+1))
predict = np.exp(etr_adv.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 18.67%"""

## DefPar_KFold - 13.59%

In [94]:
"""#данные LabelEncoding, дефолтные параметры, KFold с n_splits=4
etr_KFold = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)

skf = KFold(n_splits=4, random_state=RANDOM_SEED, shuffle=True)
etr_mape_values = []

for train_index, test_index in skf.split(X, y): # после Encoding X, после стандартизации  X_sc, после даммизации X_sc_dumm
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    
    # For training, fit() is used
    etr_KFold.fit(X_train, np.log(y_train))

    # For MAPE metric (or any other), we need the predictions of the model
    y_pred = np.exp(etr_KFold.predict(X_test))

    etr_mape_value = mape(y_test, y_pred)
    etr_mape_values.append(etr_mape_value)
    print(etr_mape_values)

print(f"Точность модели по метрике MAPE: {(np.mean(etr_mape_values) * 100):0.2f}%.")
predict_submission = np.exp(etr_KFold.predict(X_sub))

# Точность модели по метрике MAPE: 13.59%. на лидерборде 14.8318 
# Точность модели по метрике MAPE: 13.59%. после стандартизации
# Точность модели по метрике MAPE: 13.59%. после стандартизации и даммизации"""

## AdvPar_Stan_Dumm

In [95]:
"""hp.uniform('n_estimators',100,500),
hp.choice("n_estimators", [int(x) for x in np.linspace(200, 1000, num = 17)])

def objective(params):
    model=ExtraTreesRegressor(
         n_estimators=int(params['n_estimators']),
         max_depth=int(params['max_depth']),
         min_samples_leaf=int(params['min_samples_leaf']),
         min_samples_split=int(params['min_samples_split']),
         bootstrap=params['bootstrap'],
         max_features=params['max_features'],
         random_state=RANDOM_SEED,
         n_jobs=-1
     )
    model.fit(X_train, np.log(y_train))
    pred=model.predict(X_test)
    score=mape(y_test,np.exp(pred))
    return score

def optimize(trial):
     params={
         'n_estimators': hp.uniform('n_estimators',100,500),
         'max_features': hp.choice("max_features", ['auto', 'sqrt']),
         'max_depth': hp.uniform('max_depth',5,15),
         'min_samples_split': hp.uniform('min_samples_split',2,10),
         'min_samples_leaf': hp.uniform('min_samples_leaf',1,5),
         'bootstrap': hp.choice("bootstrap", [True, False])
     }
     best=fmin(fn=objective, space=params, algo=tpe.suggest, trials=trial, max_evals=100, rstate=np.random.RandomState(RANDOM_SEED))
     return best
    
trial=Trials()
best=optimize(trial)"""

Промежуточные выводы по ExtraTreesRegressor:
1. Самый начальные вариант (данные после LabelEncoding + дефолтные параметры) дал результаты 13.72
2. Стандартихзация положение не улучшило. Зато даммизирование дал пока что лучший результат.
3. При коммите на даммизированых данны + дефолтные параметры дал результат 12.391 (Точность модели по метрике MAPE: 13.5% на лидерборде 12.391)

# Model 5 - GradientBoostingRegressor

## DefPar - 18.6%

In [96]:
"""gb_def = GradientBoostingRegressor(random_state=RANDOM_SEED)
gb_def.fit(X_train, np.log(y_train+1))
predict = np.exp(gb_def.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 18.6%%"""

## AdvPar

In [97]:
"""gb_adv =  GradientBoostingRegressor(
    learning_rate = 0.1, # (усадка).
    n_estimators = 300,# (количество деревьев).
    max_depth = 8,
    min_samples_split = 3,
    min_samples_leaf = 1)
gb_adv.fit(X_train, np.log(y_train+1))
predict = np.exp(gb_adv.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")

predict_submission = np.exp(gb_adv.predict(X_sub))
# Точность модели по метрике MAPE: 12.93% на лидерборде 13.3212 min_samples_leaf = 3
# min_samples_leaf = 1 12.88% на лидерборде 13.2622"""

In [98]:
"""gb_def = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb_def.fit(X_train, np.log(y_train+1))
predict = np.exp(gb_def.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 12.93%"""

## AdvPar_Stan - 12.95% 

In [99]:
"""# данные стандартизированы, дефолтные параметры
gb_def = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb_def.fit(X_train_sc, np.log(y_train_sc+1))
predict = np.exp(gb_def.predict(X_test_sc))
result = round((mape(y_test_sc, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 12.95%"""

In [100]:
"""# данные стандартизированы + даммизирвоанные, дефолтные параметры
gb_def = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=300)
gb_def.fit(X_train_sc_dumm, np.log(y_train_sc_dumm+1))
predict = np.exp(gb_def.predict(X_test_sc_dumm))
result = round((mape(y_test_sc_dumm, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.25%%"""

# Model 6 - xgboost

## DefPar - 13.95%

In [110]:
"""xb_adv = xgb.XGBRegressor(random_state=RANDOM_SEED)
xb_adv.fit(X_train, np.log(y_train+1))
predict = np.exp(xb_adv.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")
# Точность модели по метрике MAPE: 13.95%"""

## AdvPar - 12.41% 

In [ ]:
"""xb_adv = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, \
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb_adv.fit(X_train, np.log(y_train+1))
predict = np.exp(xb_adv.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")

predict_submission = np.exp(xb_adv.predict(X_sub))
# Точность модели по метрике MAPE: 12.41%  на лидерборде 12.84127"""

In [102]:
"""
# запустим подбор параметров в помощью hyperopt
space_xgb2 = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
            'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
            'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
            'eval_metric': 'auc',
            'objective': 'binary:logistic',
            # Increase this number if you have more cores. Otherwise, remove it and it will default
            # to the maxium number.
            #'nthread': 4,
            'booster': 'gbtree',
            'tree_method': 'exact',
            'silent': 1
        }

def hyperopt_xgb_score(params):
    clf = xgb.XGBRegressor(**params)
    # усреднение по 3ем фолдам, для уменьшения влияния стахостичности
    # для ускорения можно использовать train_test_split один раз
    current_score = cross_val_score(clf, X, y, cv=3).mean()
    print(current_score, params)
    return -current_score
 
 
simple_space_xgb = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
}
 
best = fmin(fn=hyperopt_xgb_score, space=simple_space_xgb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)
"""

0.7026894762150396                                                                                                     
{'eta': 0.325, 'max_depth': 5, 'n_estimators': 812}                                                                    
0.6954513086182801                                                                                                     
{'eta': 0.30000000000000004, 'max_depth': 12, 'n_estimators': 239}   Точность модели по метрике MAPE: 13.82%                                       
0.6151461457845006                                                                                                     
{'eta': 0.2, 'max_depth': 3, 'n_estimators': 866}   Точность модели по метрике MAPE:  13.71%                                                      
0.7653448150384431                                                                                                     
{'eta': 0.275, 'max_depth': 5, 'n_estimators': 634}                                                                    
0.8041513138194616                                                                                                     
{'eta': 0.125, 'max_depth': 8, 'n_estimators': 198}  Точность модели по метрике MAPE: 12.98%                                                       
0.8028094519018834                                                                                                     
{'eta': 0.07500000000000001, 'max_depth': 8, 'n_estimators': 800}    Точность модели по метрике MAPE: 12.63%                                      
0.7623342386577633                                                                                                     
{'eta': 0.275, 'max_depth': 9, 'n_estimators': 574}                                                                    
0.7720938927893299                                                                                                     
{'eta': 0.375, 'max_depth': 8, 'n_estimators': 221}                                                                    
0.8011242940061778                                                                                                     
{'eta': 0.15000000000000002, 'max_depth': 6, 'n_estimators': 176}                                                      
0.7015653655710571                                                                                                     
{'eta': 0.42500000000000004, 'max_depth': 11, 'n_estimators': 616}                                                     
100%|██████████████████████████████████████████████| 10/10 [24:38<00:00, 147.82s/trial, best loss: -0.8041513138194616]

best:
{'eta': 0.125, 'max_depth': 7, 'n_estimators': 98} Точность модели по метрике MAPE: 14.02%

In [109]:
"""xb_adv = xgb.XGBRegressor(eta = 0.075, max_depth = 8, n_estimators = 800)
xb_adv.fit(X_train, np.log(y_train+1))
predict = np.exp(xb_adv.predict(X_test))
result = round((mape(y_test, predict) * 100),2)
print(f"Точность модели по метрике MAPE: {result}%")

predict_submission = np.exp(xb_adv.predict(X_sub))
# Точность модели по метрике MAPE: 12.74% на лидерборде 12.91952"""

# Model 7 : StackingRegressor - 12.42%

In [114]:
"""estimators = [
    ( 'etr', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
     ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000, random_state=RANDOM_SEED))
 ]

sr_log = StackingRegressor(
     estimators=estimators,
     final_estimator=LinearRegression()
 )

 # For training, fit() is used
sr_log.fit(X_train, np.log(y_train))

 # For MAPE metric (or any other), we need the predictions of the model
y_pred = np.exp(sr_log.predict(X_test))

print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred) * 100):0.2f}%.")

predict_submission = np.exp(sr_log.predict(X_sub))

# Точность модели по метрике MAPE: 12.42%. на лидерборде 12.58125"""

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing